<a href="https://colab.research.google.com/github/RichardFreedman/CRIM-Project-RF/blob/master/CRIM_Intervals_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load CRIM Intervals Software

In [ ]:
!git clone https://github.com/HCDigitalScholarship/intervals.git
!pip install httpx


## Start CRIM Intervals

In [ ]:
from intervals.main_objs import *


## Load MEI Files from CRIM or Github by pasting one or more of [these links](https://docs.google.com/spreadsheets/d/1TzRqnzgcYYuQqZR78c5nizIsBWp4pnblm2wbU03uuSQ/edit?auth_email=rfreedma@haverford.edu#gid=0) below.

*Note:  each file must be in quotation marks and separated by commas







In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei', 'https://crimproject.org/mei/CRIM_Mass_0005_5.mei'])


Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.
Requesting file from https://crimproject.org/mei/CRIM_Mass_0005_5.mei...
Successfully imported.


## Give the scores short names, in order according to the way they were listed above

In [ ]:
model, mass = corpus.scores

## Now apply various methods to the scores:
* getNoteRest returns all the notes and rests, each voice as a column
* getDuration returns the durations for all notes and rests, as above
* getMelodic returns the melodic intervals in each voice as a column
* getHarmonic returns pairs of harmonic intervals between each pair of voices
* getNgrams  returns segments of various kinds, melodic (one voice) or modular (pairs of voices, including vertical and horizontal motion)




---

### For summary statistics:
* save results as variable, then:
`.apply(pd.Series.value_counts).fillna(0).astype(int) `


---

### Documentation available via this command:
* for any method, use the following read documentation:
`print(model.getNgrams.__doc__)`

---








In [ ]:
print(model.getMelodic.__doc__)

Return melodic intervals for all voice pairs. Each melodic interval
        is associated with the starting offset of the second note in the
        interval. 

        :param str kind: use "d" for diatonic intervals without quality, "q"
            (default) for diatonic intervals with quality, or "s" for semitonal
            intervals. Only the first character is used, and it's case
            insensitive.
        :param bool directed: defaults to True which shows that the voice that
            is lower on the staff is a higher pitch than the voice that is
            higher on the staff. This is desginated with a "-" prefix.
        :param bool compound: whether to use compound (True, default) or simple
            (False) intervals. In the case of simple diatonic intervals, it
            simplifies to within the octave, so octaves don't get simplified to
            unisons. But for semitonal intervals, an interval of an octave
            (12 semitones) would does get simplifi

# Notes and Rests

In [ ]:
notes = model.getNoteRest()
notes.fillna(value= "-", inplace=True)
notes.head()

,[Superius],Altus,Tenor,Bassus
0.0,G4,Rest,Rest,Rest
4.0,C5,-,-,-
8.0,-,Rest,Rest,Rest
12.0,C5,-,-,-
16.0,D5,G3,Rest,Rest


In [ ]:
notes.value_counts()
# notes.stack().value_counts()





[Superius]  Altus  Tenor  Bassus
-           C4     -      -         23
C5          -      -      -         17
-           E4     -      -         16
B4          -      -      -         15
-           D4     -      -         14
                                    ..
D5          -      Rest   Rest       1
                   F4     D3         1
                   D4     -          1
                   B3     -          1
C5          A3     F4     F3         1
Length: 280, dtype: int64

In [ ]:
df = notes.apply(pd.Series.value_counts).fillna(0).astype(int)
df

,[Superius],Altus,Tenor,Bassus
-,271,216,279,284
A2,0,0,0,4
A3,0,21,12,18
A4,23,10,4,0
B-2,0,0,0,1
B-3,0,0,0,1
B2,0,0,0,4
B3,0,28,27,5
B4,41,1,0,0
C3,0,0,0,45


# Melodic Intervals
* kind='d' for diatonic; 's' for chromatic/semitone
* To save as CSV:  
`mel_int.to_csv('file_name.csv')`

In [ ]:
mel_int = mass.getMelodic(kind='d')
mel_int.fillna(value= "-", inplace=True)
mel_int
# mel_int.apply(pd.Series.value_counts).fillna(0).astype(int)

# voice = mel_int.iloc[:, 0]
# voice


,Sup[erius],Altus,Tenor,Bassus
0.0,-,Rest,Rest,Rest
4.0,4,-,-,-
8.0,-,Rest,Rest,Rest
12.0,1,-,-,-
16.0,2,-,Rest,Rest
...,...,...,...,...
706.0,1,-2,-,-
707.0,-,-2,-,-
708.0,-,2,-,-
710.0,-2,-,-,-


# Durations

In [ ]:
durs = mass.getDuration()
durs.fillna(value= "-", inplace=True)
d =  durs.stack().value_counts()


-       828
1.0     211
4.0     174
8.0     149
2.0     148
3.0      54
6.0      21
16.0      9
24.0      3
0.5       2
12.0      1
dtype: int64

## Combine Notes and Durations as One DataFrame

In [ ]:
notes_durs = pd.concat([notes, durs], axis=1)
notes_durs


,[Superius],Altus,Tenor,Bassus,Sup[erius],Altus,Tenor,Bassus
0.0,G4,Rest,Rest,Rest,4,8,8,8
4.0,C5,-,-,-,8,-,-,-
8.0,-,Rest,Rest,Rest,-,8,8,8
12.0,C5,-,-,-,4,-,-,-
16.0,D5,G3,Rest,Rest,4,4,8,8
...,...,...,...,...,...,...,...,...
1256.0,-,-,D4,G3,NaN,NaN,NaN,NaN
1268.0,C5,G4,-,-,NaN,NaN,NaN,NaN
1272.0,-,-,C4,C3,NaN,NaN,NaN,NaN
1284.0,C5,G4,-,-,NaN,NaN,NaN,NaN


# Select Columns for One Voice

In [ ]:
notes_durs_s = notes_durs.iloc[:, [0,4]]
notes_durs_s

,[Superius],Sup[erius]
0.0,G4,4
4.0,C5,8
8.0,-,-
12.0,C5,4
16.0,D5,4
...,...,...
1256.0,-,NaN
1268.0,C5,NaN
1272.0,-,NaN
1284.0,C5,NaN


## N-Grams
* for Melodic or Durations

In [ ]:
mel = model.getMelodic()
model.getNgrams(df=mel, n=3)





,[Superius],Altus,Tenor,Bassus
"-M2, -M2, -M2",3,0,0,0
"-M2, -M2, -m2",9,7,6,2
"-M2, -M2, M2",3,4,3,3
"-M2, -M2, M3",0,0,0,1
"-M2, -M2, P1",3,2,2,1
...,...,...,...,...
"m3, -M2, -m2",2,0,2,0
"m3, -m2, -M2",0,0,1,0
"m3, -m2, P1",0,1,0,0
"m3, M2, -M2",0,1,0,0


# Two-Voice Modules as N-Grams

In [ ]:
modules = model.getNgrams(how='modules', cell_type=str)
mod = modules.iloc[:, 0:6]
# mod.apply(pd.Series.value_counts).fillna(0).astype(int)


,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
"-2_2, -3_2, -5",0,0,0,1,0,0
"-2_2, -3_Held, -2",0,2,0,0,0,0
"-2_2, -3_Held, -3",0,0,0,2,0,0
"-2_2, 3_Held, 3",0,2,0,0,0,0
"-2_Held, -3_Held, -3",0,0,0,4,0,0
...,...,...,...,...,...,...
"9_Held, 8_Held, 8",0,2,0,0,0,0
"9_Held, 9_Held, 10",0,0,2,0,0,0
"9_Rest, 10_Held, 9",2,0,0,0,0,0
"9_Rest, 8_Held, 6",0,1,0,0,0,0
